# Keras Tuner | Hyperparameter Tuning a Neural Network


In [1]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [2]:
!pip install keras_tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import keras_tuner as kt

In [4]:
!wget https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/diabetes_1.csv

--2024-01-30 22:10:03--  https://frenzy86.s3.eu-west-2.amazonaws.com/python/data/diabetes_1.csv
Resolving frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)... 3.5.246.13, 3.5.244.11, 52.95.149.122, ...
Connecting to frenzy86.s3.eu-west-2.amazonaws.com (frenzy86.s3.eu-west-2.amazonaws.com)|3.5.246.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23873 (23K) [text/csv]
Saving to: ‘diabetes_1.csv’

diabetes_1.csv      100%[===================>]  23.31K  --.-KB/s    in 0.1s    

2024-01-30 22:10:03 (175 KB/s) - ‘diabetes_1.csv’ saved [23873/23873]



In [5]:
df = pd.read_csv('diabetes_1.csv')
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [7]:
# There is two type of datatype:
# float64(2), int64(7)

In [8]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [9]:
df.shape

(768, 9)

In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [11]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [12]:
# let's Check Null values
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [13]:
# there is no Null values

In [14]:
df.duplicated().value_counts()

False    768
dtype: int64

In [15]:
# There is no duplicated

In [16]:
# let's extract the columns
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [17]:
X.shape, y.shape

((768, 8), (768,))

In [18]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [19]:
# let's scaling X data mean feature data

In [20]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [21]:
X

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [22]:
X.shape

(768, 8)

In [23]:
# lets' splitting the data

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=667
                                                    )

# Modeling with keras Tuner


In [25]:
# Normal model witout keras_tuner
model = Sequential()

model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', ))
model.add(Dense(1, activation='sigmoid')) #

model.compile(optimizer='Adam',loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(X_test, y_test)
          )

Epoch 1/10
20/20 [==============================] - 1s 10ms/step - loss: 0.6863 - accuracy: 0.5717 - val_loss: 0.6534 - val_accuracy: 0.5649
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6351 - accuracy: 0.6515 - val_loss: 0.6171 - val_accuracy: 0.5909
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5965 - accuracy: 0.6743 - val_loss: 0.5903 - val_accuracy: 0.6364
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5674 - accuracy: 0.7117 - val_loss: 0.5712 - val_accuracy: 0.6883
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5451 - accuracy: 0.7199 - val_loss: 0.5587 - val_accuracy: 0.7078
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5273 - accuracy: 0.7313 - val_loss: 0.5506 - val_accuracy: 0.7078
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5137 - accuracy: 0.7410 - val_loss: 0.5439 - val_accuracy: 0.7078
Epoch 8/10
20/20 [=

In [27]:
# val_accuracy: 0.7208

In [28]:
# let's build Model with keras_tuner

def build_model(hp):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = hp.Choice('optimizer',
                          values=['rmsprop', 'adam', 'sgd', 'adadelta'])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [29]:
tuner = kt.RandomSearch(
                        build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='optimizerTuning',
                        )

In [30]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             )

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.4740259647369385

Best val_accuracy So Far: 0.7272727489471436
Total elapsed time: 00h 00m 09s


In [31]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [32]:
model = tuner.get_best_models(num_models=1)[0]

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321 (1.25 KB)
Trainable params: 321 (1.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [34]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=100,
          initial_epoch=6,
          validation_data=(X_test, y_test)
          )

Epoch 7/100
20/20 [==============================] - 1s 12ms/step - loss: 0.4932 - accuracy: 0.7736 - val_loss: 0.5553 - val_accuracy: 0.7273
Epoch 8/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4834 - accuracy: 0.7785 - val_loss: 0.5508 - val_accuracy: 0.7208
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4752 - accuracy: 0.7769 - val_loss: 0.5470 - val_accuracy: 0.7338
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4686 - accuracy: 0.7769 - val_loss: 0.5440 - val_accuracy: 0.7273
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4632 - accuracy: 0.7850 - val_loss: 0.5428 - val_accuracy: 0.7273
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4586 - accuracy: 0.7899 - val_loss: 0.5415 - val_accuracy: 0.7208
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4551 - accuracy: 0.7915 - val_loss: 0.5411 - val_accuracy: 0.7273
Epoch 14

In [35]:
# After optimizer tuner val_accuracy: 0.6883

In [36]:
# let's tuning neuron
def build_model(hp):
    model = Sequential()
    # Tune the number of units in the first Dense layer
    units = hp.Int('units',min_value=8, max_value=128, step=8)
    model.add(Dense(units=units,activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [37]:
tuner = kt.RandomSearch(
                        build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory='NeuronTuning',
                        )

In [38]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             )

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7337662577629089

Best val_accuracy So Far: 0.7337662577629089
Total elapsed time: 00h 00m 06s


In [39]:
tuner.get_best_hyperparameters()[0].values

{'units': 56}

In [40]:
model = tuner.get_best_models(num_models=1)[0]

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 56)                504       
                                                                 
 dense_1 (Dense)             (None, 1)                 57        
                                                                 
Total params: 561 (2.19 KB)
Trainable params: 561 (2.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [42]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=100,
          initial_epoch=6,
          validation_data=(X_test, y_test)
          )

Epoch 7/100
20/20 [==============================] - 1s 13ms/step - loss: 0.5074 - accuracy: 0.7704 - val_loss: 0.5361 - val_accuracy: 0.7403
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4873 - accuracy: 0.7915 - val_loss: 0.5289 - val_accuracy: 0.7208
Epoch 9/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4738 - accuracy: 0.7866 - val_loss: 0.5241 - val_accuracy: 0.7273
Epoch 10/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4637 - accuracy: 0.7899 - val_loss: 0.5213 - val_accuracy: 0.7208
Epoch 11/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4570 - accuracy: 0.7948 - val_loss: 0.5213 - val_accuracy: 0.7273
Epoch 12/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4521 - accuracy: 0.8046 - val_loss: 0.5244 - val_accuracy: 0.7143
Epoch 13/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4477 - accuracy: 0.7964 - val_loss: 0.5243 - val_accuracy: 0.7273
Epoch 14

In [43]:
# after neuron tuning it's improved have a look: val_accuracy: 0.7078

### Let's Tuning All Togather


In [44]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(units=hp.Int('units', min_value=8, max_value=128, step=8),
                                 activation='relu', input_dim=8))
    model.dropout = hp.Float('dropout', min_value=0.1, max_value=0.9, step=0.1)
    for i in range(hp.Int('num_layers', 1, 10)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                                  min_value=8,
                                                  max_value=128,
                                                  step=8),
                                     activation=hp.Choice('activation_' + str(i),
                                                          values=['relu', 'tanh', 'sigmoid']),
                                     input_dim=8))
        # model.add(Dropout(hp.Choice('dropout' + str(i),
                                                #  values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        optimizer=(hp.Choice('optimizer',
                             values=['sgd', 'rmsprop', 'adam', 'nadam', 'adadelta'])),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [45]:
tuner = kt.RandomSearch(
                        build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='FinalTune',
                        project_name='Final_Tune'
                        )

In [46]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test)
             )

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.6363636255264282

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 07s


In [47]:
tuner.get_best_hyperparameters()[0].values

{'units': 24,
 'dropout': 0.8,
 'num_layers': 10,
 'units_0': 56,
 'activation_0': 'sigmoid',
 'optimizer': 'sgd',
 'units_1': 8,
 'activation_1': 'relu',
 'units_2': 8,
 'activation_2': 'relu',
 'units_3': 8,
 'activation_3': 'relu',
 'units_4': 8,
 'activation_4': 'relu',
 'units_5': 8,
 'activation_5': 'relu',
 'units_6': 8,
 'activation_6': 'relu',
 'units_7': 8,
 'activation_7': 'relu',
 'units_8': 8,
 'activation_8': 'relu',
 'units_9': 8,
 'activation_9': 'relu'}

In [48]:
model = tuner.get_best_models(num_models=1)[0]

In [49]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                216       
                                                                 
 dense_1 (Dense)             (None, 56)                1400      
                                                                 
 dense_2 (Dense)             (None, 8)                 456       
                                                                 
 dense_3 (Dense)             (None, 8)                 72        
                                                                 
 dense_4 (Dense)             (None, 8)                 72        
                                                                 
 dense_5 (Dense)             (None, 8)                 72        
                                                                 
 dense_6 (Dense)             (None, 8)                 7

In [50]:
model.fit(X_train, y_train,
          batch_size=32,
          epochs=100,
          initial_epoch=6,
          validation_data=(X_test, y_test)
          )

Epoch 7/100
20/20 [==============================] - 1s 11ms/step - loss: 0.6861 - accuracy: 0.6564 - val_loss: 0.6850 - val_accuracy: 0.6364
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6824 - accuracy: 0.6547 - val_loss: 0.6817 - val_accuracy: 0.6364
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6788 - accuracy: 0.6547 - val_loss: 0.6785 - val_accuracy: 0.6364
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6752 - accuracy: 0.6547 - val_loss: 0.6758 - val_accuracy: 0.6364
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6721 - accuracy: 0.6547 - val_loss: 0.6733 - val_accuracy: 0.6364
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6693 - accuracy: 0.6547 - val_loss: 0.6714 - val_accuracy: 0.6364
Epoch 13/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6669 - accuracy: 0.6547 - val_loss: 0.6692 - val_accuracy: 0.6364
Epoch 14